<a href="https://colab.research.google.com/github/AjayrajSingh/AjayrajSingh/blob/main/DatasetDownloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium

     |████████████████████████████████| 911kB 6.5MB/s 


In [ ]:
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:12 http://security.ubuntu.com/ub

In [ ]:
import sys
from google.colab import drive

drive.mount('/content/drive')
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

Mounted at /content/drive


In [ ]:
import selenium
from selenium import webdriver
import time
import requests
import os
from PIL import Image
import io
import hashlib

DRIVER_PATH = './chromedriver'


def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        results_start = len(thumbnail_results)

    return image_urls

def persist_image(folder_path:str,file_name:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        folder_path = os.path.join(folder_path,file_name)
        if os.path.exists(folder_path):
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        else:
            os.mkdir(folder_path)
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

if __name__ == '__main__':
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
    queries = ["Parivrtta Trikonasana","Parivrtta Utkatasana","Tadasana","Urdvha Hastasana","Uttanasana","Ardha Uttanasana","Chaturanga Dandasana","Virabhadrasana-I"]
    for query in queries:
        wd.get('https://google.com')
        search_box = wd.find_element_by_css_selector('input.gLFyf')
        search_box.send_keys(query)
        links = fetch_image_urls(query,150,wd)
        images_path = '/content/drive/MyDrive/Project Yogic/dataset'
        for i in links:
            print(images_path)
            persist_image(images_path,query,i)
    wd.quit()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:89: DeprecationWarning: use options instead of chrome_options


Found: 100 search results. Extracting links from 0:100
Found: 150 image links, done!
/content/drive/MyDrive/Project Yogic/dataset
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSiWLkVLyux3oUp6tbaAjFnU5pB-jlqK0GiCg&usqp=CAU - as /content/drive/MyDrive/Project Yogic/dataset/Parivrtta Trikonasana/d822c47442.jpg
/content/drive/MyDrive/Project Yogic/dataset
SUCCESS - saved https://images.squarespace-cdn.com/content/v1/5372014be4b0db8de8ce9150/1538841105654-3GT8I785BF9AGAWWPK5Q/ke17ZwdGBToddI8pDm48kFWxnDtCdRm2WA9rXcwtIYR7gQa3H78H3Y0txjaiv_0fDoOvxcdMmMKkDsyUqMSsMWxHk725yiiHCCLfrh8O1z5QPOohDIaIeljMHgDF5CVlOqpeNLcJ80NK65_fV7S1UcTSrQkGwCGRqSxozz07hWZrYGYYH8sg4qn8Lpf9k1pYMHPsat2_S1jaQY3SwdyaXg/IMG_0480.jpg - as /content/drive/MyDrive/Project Yogic/dataset/Parivrtta Trikonasana/feb12942a7.jpg
/content/drive/MyDrive/Project Yogic/dataset
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQVuL7BWtVwLDJQfT8qxKF2MXJAtJvB2N7AxQ&usqp=CAU - as /content/drive/MyDr

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


SUCCESS - saved https://tummeeyoga.s3.amazonaws.com/revolved_triangle_pose_block__parivrtta_trikonasana_block_yoga.png - as /content/drive/MyDrive/Project Yogic/dataset/Parivrtta Trikonasana/c30e7d6727.jpg
/content/drive/MyDrive/Project Yogic/dataset
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTrYzQGlOmxqKVD08PDNymFHJ8tCkWxobI_zQ&usqp=CAU - as /content/drive/MyDrive/Project Yogic/dataset/Parivrtta Trikonasana/7a15356665.jpg
/content/drive/MyDrive/Project Yogic/dataset
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQpCXwlV2exV8fulLtfuQ2sF4-AQaJggcDwlg&usqp=CAU - as /content/drive/MyDrive/Project Yogic/dataset/Parivrtta Trikonasana/d8280ca1c3.jpg
/content/drive/MyDrive/Project Yogic/dataset
ERROR - Could not save https://www.indianyogaassociation.com/blog/images/strap-to-open-shoulder.jpg - cannot identify image file <_io.BytesIO object at 0x7feda26d5ca8>
/content/drive/MyDrive/Project Yogic/dataset
SUCCESS - saved https://www.ayurvedam.se